In [30]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional

class ArtifactFile(BaseModel):
    path: str = Field(..., description="Relative file path")
    template: str = Field(..., description="Exact file contents as a template string")

class ArtifactAsset(BaseModel):
    path: str = Field(..., description="Relative asset path")
    download_url: str = Field(..., description="URL to fetch the asset from")

class ArtifactSpec(BaseModel):
    artifact_id: str = Field(..., description="Unique ID for the artifact")
    title: str = Field(..., description="Human-readable title")
    dependencies: Dict[str, str] = Field(default_factory=dict, description="package.json dependencies")
    devDependencies: Dict[str, str] = Field(default_factory=dict, description="package.json devDependencies")
    scripts: Dict[str, str] = Field(default_factory=dict, description="package.json scripts")
    files: List[ArtifactFile] = Field(default_factory=list, description="files to create with templates")
    assets: Optional[List[ArtifactAsset]] = Field(default=None, description="optional assets to download")




def get_spec_creator_prompt(user_request: str) -> str:
    """
    Returns a system prompt instructing the Spec-Creator agent to output
    a JSON dictionary matching the ArtifactSpec schema.
    """
    return f'''You are Bolt’s Spec-Creator.
You will receive a user request describing an app to build.
Your job is to output exactly one Python dictionary `spec` that matches this Pydantic schema:


Instructions:
1. Do not emit any markdown or commentary, only valid Python dictionary literal.
2. All keys must appear exactly as in the schema.
3. Use sensible defaults inferred from the user request.
4. Omit `assets` if none are needed.

User request:
"""
{user_request}
"""'''  

def get_creator_prompt(spec, cwd: str = '.', tools: list = [], ui_components: list = None) -> str:
    """
    Returns the system prompt for the Creator agent.
    It uses the provided `spec` dictionary to generate a <boltArtifact>.
    """
    tools_list = "\n".join(f"- {t.name}: {t.description}" for t in tools)
    ui_list = "" if not ui_components else "\nAvailable UI components:\n" + "\n".join(f"- {c}" for c in ui_components)

    return f"""
You are Bolt, a senior software engineer and expert assistant.  
Your job is to take the following `spec` and emit one `<boltArtifact>`:

```python
spec = {spec}
```

<artifact_info>
  1. Wrap everything in `<boltArtifact id=\"{spec['artifact_id']}\" title=\"{spec['title']}\">…</boltArtifact>`.
  2. Use only `<boltAction type=\"shell\">COMMAND</boltAction>` or `<boltAction type=\"file\" filePath=\"PATH\">CONTENT</boltAction>`.
  3. Do not chain shell commands.
  4. Include full file contents; no placeholders.
  5. Do not escape `<`, `>`, `&`, or quotes.
  6. Final shell action must run `{spec['scripts'].get('dev','npm run dev')}`.
</artifact_info>

<system_constraints>
  • Environment: WebContainer (Node.js + zsh).
  • No git, no native binaries.
  • Use pure JS/WebAssembly packages.
</system_constraints>

<generation_logic>
1. Create `package.json` from `spec.dependencies`, `spec.devDependencies`, `spec.scripts`.
2. For each item in `spec.files`, emit a file action.
3. For each item in `spec.assets` (if any), emit a shell action to `curl`.
4. Emit `<boltAction type=\"shell\">npm install</boltAction>` then `<boltAction type=\"shell\">{spec['scripts'].get('dev','npm run dev')}</boltAction>`.
5. Wrap all in `<boltArtifact>`.
</generation_logic>

<validator>
• Confirm package.json matches spec.
• One action per file/asset.
• Final action runs dev script.
</validator>
{ui_list}
"""


In [31]:
from dotenv import load_dotenv
import os
load_dotenv()

model_name="gpt-4.1-nano"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name=model_name,
    temperature=0,
    streaming=True,
    verbose=True,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)
# Set up the language model with streaming
structured_llm = llm.with_structured_output(ArtifactSpec)



In [32]:
# Bind the spec creator to produce JSON
spec_model = llm.with_structured_output(
    schema=ArtifactSpec,
    method="function_calling",
)
spec = spec_model.invoke("Build a todo app in React using TailwindCSS.")

In [34]:
print(spec)

artifact_id='react-todo-app' title='React Todo App with TailwindCSS' dependencies={} devDependencies={} scripts={} files=[ArtifactFile(path='src/App.jsx', template='import { useState } from \'react\';\n\nfunction App() {\n  const [todos, setTodos] = useState([]);\n  const [input, setInput] = useState(\'\');\n\n  const addTodo = () => {\n    if (input.trim() !== \'\') {\n      setTodos([...todos, { text: input, completed: false }]);\n      setInput(\'\');\n    }\n  };\n\n  const toggleComplete = (index) => {\n    const newTodos = [...todos];\n    newTodos[index].completed = !newTodos[index].completed;\n    setTodos(newTodos);\n  };\n\n  const deleteTodo = (index) => {\n    const newTodos = [...todos];\n    newTodos.splice(index, 1);\n    setTodos(newTodos);\n  };\n\n  return (\n    <div className="min-h-screen bg-gray-100 flex items-center justify-center p-4">\n      <div className="bg-white rounded-lg shadow-lg p-6 max-w-md w-full">\n        <h1 className="text-2xl font-bold mb-4 text-